In [1]:
import numpy as np
import pandas as pd
import re
import nltk
df = pd.read_csv("AI_CRJ.csv",header=0)

In [2]:
df1 = pd.read_csv("FI_CRJ.csv",header=0)

D:\Programs\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (15,16,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df1 = df1[['ATA_SYMPTOM', 'ATA_CAUSE', 'INTERRUPTION_REASON', 'CORRECTIVE_ACTION']]
df = df[['ATA_SYMPTOM', 'ATA_CAUSE', 'INTERRUPTION_REASON', 'CORRECTIVE_ACTION']]
df2 = df.append(df1, ignore_index=True)

In [4]:
df2['ATA_SYM_4'] = df2['ATA_SYMPTOM'].astype(str).str[:4]

In [5]:
df2['ATA_CAUSE_4'] = df2['ATA_CAUSE'].astype(str).str[:4]

In [6]:
df2['features'] = df2['INTERRUPTION_REASON'].str.cat(df2['CORRECTIVE_ACTION'], sep='  ',na_rep=' ')

In [7]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
def identify_tokens1(df):
    comment = df['features']
    comment = re.sub(r'\W',' ',comment)
    comment = re.sub(r'\s+',' ', comment)
    words = nltk.word_tokenize(comment)
    new_words = [stemmer.stem(word) for word in words]
    comment = ' '.join(new_words)
    return comment

df2['features_1'] = df2.apply(identify_tokens1, axis=1)

ATA_SYMPTOM

In [8]:
df2_ATA_SYM = df2.dropna(axis=0, subset=['ATA_SYMPTOM'])

In [9]:
len(df2_ATA_SYM)

401204

In [23]:
from sklearn.model_selection import train_test_split
X = df2_ATA_SYM['features_1']
y = df2_ATA_SYM['ATA_SYMPTOM']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('tfidf',TfidfVectorizer(sublinear_tf=False,ngram_range=(1,2),stop_words='english')),('clf',CalibratedClassifierCV(LinearSVC()))])
text_clf.fit(X_train,y_train)

D:\Programs\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=9000, stop_words='english')),
                ('clf', CalibratedClassifierCV(base_estimator=LinearSVC()))])

In [25]:
from sklearn import metrics
predictions = text_clf.predict(X_test)
metrics.accuracy_score(y_test,predictions)

0.6678796640163506

In [29]:
import joblib
joblib.dump(text_clf, 'crj_all_ATA_SYM.pkl')

['crj_all_ATA_SYM.pkl']

In [26]:
predictions_prob = pd.DataFrame(text_clf.predict_proba(X_test),columns=text_clf.classes_)

In [27]:
ata_sym_top3 = lambda row, nlargest=3:sorted(pd.Series(zip(predictions_prob.columns, row)), key=lambda cv: -cv[1]) [:nlargest]

tmp = predictions_prob.apply(ata_sym_top3, axis=1)
result = pd.DataFrame(list(map(np.ravel, tmp)))
result

,0,1,2,3,4,5
0,4960.0,0.550742,4914.0,0.155903,1200.0,0.106116
1,2561.0,0.535825,256107.0,0.308014,256105.0,0.053452
2,200.0,0.803827,2911.0,0.027084,7200.0,0.008117
3,3461.0,0.874044,346001.0,0.013513,3245.0,0.002452
4,561201.0,0.511753,5612.0,0.136925,2526.0,0.038714
...,...,...,...,...,...,...
40116,8011.0,0.297017,8010.0,0.245483,3611.0,0.029079
40117,221201.0,0.384616,100.0,0.364516,2212.0,0.028306
40118,273121.0,0.593249,2731.0,0.095401,2735.0,0.020549
40119,3512.0,0.784146,3510.0,0.109338,1200.0,0.007731


In [28]:
result.to_csv("temp.csv",index=False,quoting=3,sep=',',escapechar='\\')
output=pd.DataFrame(data={"Real":y_test,"prediction":predictions,"Features_Preprocessed":X_test}) 
#output
output.to_csv("Prediction_ALL_ATA_SYM_CRJ_TOP3_0415.csv",index=False,quoting=3,sep=',',escapechar='\\')

ATA_CAUSE

In [30]:
from sklearn.model_selection import train_test_split
X = df2['features_1']
y = df2['ATA_CAUSE']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('tfidf',TfidfVectorizer(sublinear_tf=False,ngram_range=(1,2),stop_words='english')),('clf',CalibratedClassifierCV(LinearSVC()))])
text_clf.fit(X_train,y_train)

D:\Programs\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=9000, stop_words='english')),
                ('clf', CalibratedClassifierCV(base_estimator=LinearSVC()))])

In [32]:
predictions = text_clf.predict(X_test)
metrics.accuracy_score(y_test,predictions)

0.7389200925095577

In [33]:
joblib.dump(text_clf, 'crj_all_ATA_CAUSE.pkl')
predictions_prob = pd.DataFrame(text_clf.predict_proba(X_test),columns=text_clf.classes_)

In [34]:
ata_sym_top3 = lambda row, nlargest=3:sorted(pd.Series(zip(predictions_prob.columns, row)), key=lambda cv: -cv[1]) [:nlargest]

tmp = predictions_prob.apply(ata_sym_top3, axis=1)
result = pd.DataFrame(list(map(np.ravel, tmp)))
result

,0,1,2,3,4,5
0,4920.0,0.340397,4960.0,0.169046,491402.0,0.148646
1,253202.0,0.893580,252111.0,0.009457,2532.0,0.003180
2,324201.0,0.912251,215104.0,0.002116,262102.0,0.001711
3,2735.0,0.636012,273501.0,0.032134,3244.0,0.026621
4,2322.0,0.307594,200.0,0.111487,276105.0,0.051855
...,...,...,...,...,...,...
42369,200.0,0.864715,2741.0,0.005121,2911.0,0.005051
42370,200.0,0.278328,2761.0,0.164302,7930.0,0.056693
42371,7112.0,0.196574,2912.0,0.104784,2515.0,0.091490
42372,215101.0,0.742055,2151.0,0.072435,4920.0,0.035688


In [35]:
result.to_csv("temp.csv",index=False,quoting=3,sep=',',escapechar='\\')
output=pd.DataFrame(data={"Real":y_test,"prediction":predictions,"Features_Preprocessed":X_test}) 
#output
output.to_csv("Prediction_ALL_ATA_CAUSE_digits_CRJ_TOP3_0415.csv",index=False,quoting=3,sep=',',escapechar='\\')